# 0 Collecting airline data from 2011 to 2015

Since the whole year data of 2016 is not avalible online. We decide to use the data from 2011 to 2015 to building perdiction models.

And we conclude <b>US Official Flight Data</b>, <b>Geometric Information of Airport</b> and <b>Aircraft Information</b> in our final dataset.

<b>US Official Flight Data</b>:<a href="http://www.transtats.bts.gov/Tables.asp?DB_ID=120&DB_Name=Airline%20On-Time%20Performance%20Data&DB_Short_Name=On-Time">http://www.transtats.bts.gov/Tables.asp?DB_ID=120&DB_Name=Airline%20On-Time%20Performance%20Data&DB_Short_Name=On-Time</a></br>

<b>Geometric Information of Airport</b>:<a href=" http://openflights.org/data.html"> http://openflights.org/data.html</a>

<b>Aircraft Information</b>:<a href=" http://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/"> http://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/</a>

In [1]:
# import required modules for prediction tasks
import numpy as np
import pandas as pd
import math
import random
import requests
import zipfile
from io import StringIO
import re
import json
import os

In [2]:
def filterDF(df, cols):
    colsToKeep = list(set(df.columns) & set(cols))
    
    return df[colsToKeep]

In [3]:
def compressManufacturers(df, percentage=1.):
    df['AIRCRAFT_MFR'] = df['AIRCRAFT_MFR'].map(lambda x: x.strip())
    mfr_stats = df['AIRCRAFT_MFR'].value_counts()
    
    market_share = mfr_stats.values * 100. / np.sum(mfr_stats.values)
    idxs = np.where(market_share < percentage)
    names = np.array([el for el in list(mfr_stats.keys())])

    # get labels for small manufacturers
    smallMFR = names[idxs]

    # perform merging for the big companies
    # Douglas airplanes
    df.loc[df['AIRCRAFT_MFR'] == 'MCDONNELL DOUGLAS AIRCRAFT CO', 'AIRCRAFT_MFR'] = 'MCDONNELL DOUGLAS'
    df.loc[df['AIRCRAFT_MFR'] == 'MCDONNELL DOUGLAS CORPORATION', 'AIRCRAFT_MFR'] = 'MCDONNELL DOUGLAS'
    df.loc[df['AIRCRAFT_MFR'] == 'MCDONNELL DOUGLAS CORPORATION', 'AIRCRAFT_MFR'] = 'DOUGLAS'

    # Embraer
    df.loc[df['AIRCRAFT_MFR'] == 'EMBRAER S A', 'AIRCRAFT_MFR'] = 'EMBRAER'

    # Airbus
    df.loc[df['AIRCRAFT_MFR'] == 'AIRBUS INDUSTRIE', 'AIRCRAFT_MFR'] = 'AIRBUS'

    # the small manufacturers
    for name in smallMFR:
        df.loc[df['AIRCRAFT_MFR'] == name, 'AIRCRAFT_MFR'] = 'SMALL'
        
    return df



In [4]:
print('loading helper files...')
# load helper files
z = zipfile.ZipFile('AircraftInformation.zip')
df_master  = pd.DataFrame.from_csv(z.open('MASTER.txt'))
df_aircrafts  = pd.DataFrame.from_csv(z.open('ACFTREF.txt'))
master = df_master[['MFR MDL CODE', 'YEAR MFR']].reset_index()
aircrafts = df_aircrafts['MFR'].reset_index()
master.columns = ['TailNum', 'CODE', 'YEAR']
aircrafts.columns = ['CODE', 'MFR']
joinedAircraftInfo = pd.merge(master, aircrafts, how='left', on='CODE')
joinedAircraftInfo.TAIL_NUM = joinedAircraftInfo.TailNum.apply(lambda x: x.strip())

loading helper files...


In [5]:
columnsToUse = [u'Year', u'Quarter', u'Month', u'DayofMonth', u'DayOfWeek', u'DestCityName', u'OriginCityName'
       u'FlightDate', u'UniqueCarrier', u'AirlineID',u'TailNum',
       u'FlightNum', u'Origin', u'OriginCityName',
       u'OriginStateName', u'OriginWac', u'Dest',
       u'DestCityName',u'ArrDelay', u'ArrDelayMinutes',
       u'ArrDel15', u'Cancelled', u'Diverted', u'Distance',u'AIRCRAFT_YEAR', u'AIRCRAFT_AGE',
       u'AIRCRAFT_MFR', u'ArrTime', u'DepTime']

In [6]:
def processData(rawData):
    # filter to exclude diverted and cancelled flights
    filteredData = rawData[(rawData.Diverted == 0) & (rawData.Cancelled == 0)]

    # this is how much percent have been cleaned away!
    cleaned_away = filteredData.count()[0]

    # remove columns that are not needed for the model
    filteredData = filterDF(filteredData, columnsToUse)
    filteredData.reset_index(inplace=True)

    # perform as next step join to amend information by aircraftdata
    delayFinal = filteredData[['TailNum','UniqueCarrier']]
    delayFinal.TailNum = delayFinal.TailNum.str.strip('N')
    delaymfr = pd.merge(delayFinal, joinedAircraftInfo, how='left', on=['TailNum'])
    filteredData.TailNum = delaymfr.TailNum
    filteredData['AIRCRAFT_YEAR'] = delaymfr.YEAR
    filteredData['AIRCRAFT_MFR'] = delaymfr.MFR

    # get rid of NAN values
    filteredData.dropna(axis = 0, inplace = True)

    # get rid of empty year values
    filteredData = filteredData[filteredData['AIRCRAFT_YEAR'] != '    ']

    # compute age of aircraft
    filteredData['AIRCRAFT_AGE'] = filteredData.Year.astype(int) - filteredData.AIRCRAFT_YEAR.astype(int)

    # now, compress manufacturers to only a small amount of companies
    filteredData = compressManufacturers(filteredData)

    cleaned_away = 1. - filteredData.count()[0] * 1. / cleaned_away
    return filteredData, cleaned_away


In [15]:
bigdf = None
ca_statistic = []

years = ['2011']
months = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
print('starting processing')
for y in years:
    for m in months:
        
        print ('reading On_Time_On_Time_Performance_{y}_{mo}.zip'.format(y=y, mo = m))
        z = zipfile.ZipFile('dataset/On_Time_On_Time_Performance_{y}_{mo}.zip'.format(y=y, mo = m))
        rawData = pd.read_csv(z.open(z.namelist()[0]), low_memory=False)
        #rawData.columns = []

        print ('On_Time_On_Time_Performance_{y}_{mo}.zip'.format(y=y, mo = m))
        df, ca = processData(rawData)
        if bigdf is None:
            bigdf = df
        else:
            bigdf = bigdf.append(df, ignore_index=True)
        ca_statistic.append(('On_Time_On_Time_Performance_{y}_{mo}.zip'.format(y=y, mo = m), ca))
        print ('==> cleaned away {pc}%'.format(pc=ca))
        print ('==> added entries: {ne}'.format(ne=df.count()[0]))

starting processing
reading On_Time_On_Time_Performance_2011_1.zip
On_Time_On_Time_Performance_2011_1.zip


/Applications/Anaconda/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


==> cleaned away 0.32388857731993537%
==> added entries: 320681
reading On_Time_On_Time_Performance_2011_2.zip
On_Time_On_Time_Performance_2011_2.zip
==> cleaned away 0.3129746974544765%
==> added entries: 296966
reading On_Time_On_Time_Performance_2011_3.zip
On_Time_On_Time_Performance_2011_3.zip
==> cleaned away 0.31552330567681386%
==> added entries: 355151
reading On_Time_On_Time_Performance_2011_4.zip
On_Time_On_Time_Performance_2011_4.zip
==> cleaned away 0.31861529889614737%
==> added entries: 337528
reading On_Time_On_Time_Performance_2011_5.zip
On_Time_On_Time_Performance_2011_5.zip
==> cleaned away 0.31684712899646683%
==> added entries: 347263
reading On_Time_On_Time_Performance_2011_6.zip
On_Time_On_Time_Performance_2011_6.zip
==> cleaned away 0.3144746651833138%
==> added entries: 354928
reading On_Time_On_Time_Performance_2011_7.zip
On_Time_On_Time_Performance_2011_7.zip
==> cleaned away 0.31299306539407945%
==> added entries: 368538
reading On_Time_On_Time_Performance_20

In [16]:
# save to csv
bigdf.to_csv('FlightTable2011.csv')

In [17]:
docName = ['FlightTable2011', 'FlightTable2012','FlightTable2013', 'FlightTable2014','FlightTable2015']

In [18]:
filename = []
for i in range(len(docName)):
    filename.append(docName[i]+'.csv')

In [19]:
import pandas as pd
df_list = []
for i in range(len(docName)):
    df_tmp = pd.read_csv(filename[i])
    df_list.append(df_tmp)

In [20]:
ret = pd.concat(df_list).reset_index()
del ret['index']

In [23]:
ret.head()

,level_0,Unnamed: 0,TailNum,DayofMonth,Cancelled,ArrTime,Year,Month,OriginCityName,FlightNum,...,UniqueCarrier,Distance,ArrDelay,OriginStateName,DestCityName,Dest,ArrDel15,AIRCRAFT_YEAR,AIRCRAFT_MFR,AIRCRAFT_AGE
0,0,0,339AA,2,0.0,1236.0,2011,1,"New York, NY",1,...,AA,2475.0,1.0,New York,"Los Angeles, CA",LAX,0.0,1988,BOEING,23
1,1,1,332AA,3,0.0,1239.0,2011,1,"New York, NY",1,...,AA,2475.0,4.0,New York,"Los Angeles, CA",LAX,0.0,1987,BOEING,24
2,2,2,335AA,4,0.0,1217.0,2011,1,"New York, NY",1,...,AA,2475.0,-18.0,New York,"Los Angeles, CA",LAX,0.0,1987,BOEING,24
3,3,3,335AA,5,0.0,1214.0,2011,1,"New York, NY",1,...,AA,2475.0,-21.0,New York,"Los Angeles, CA",LAX,0.0,1987,BOEING,24
4,4,4,338AA,6,0.0,1351.0,2011,1,"New York, NY",1,...,AA,2475.0,76.0,New York,"Los Angeles, CA",LAX,1.0,1987,BOEING,24


In [30]:
ret.drop(['Unnamed: 0','level_0'],axis=1)

,Origin,ArrDel15,OriginCityName,AirTime,DayOfWeek,OriginStateName,ArrDelayMinutes,Year,DayofMonth,DepTime,...,ArrDelay,Month,UniqueCarrier,Diverted,Dest,DestCityName,AirlineID,AIRCRAFT_YEAR,AIRCRAFT_MFR,AIRCRAFT_AGE
0,JFK,0.0,"New York, NY",362.0,7,New York,1.0,2011,2,908.0,...,1.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1988,BOEING,23
1,JFK,0.0,"New York, NY",359.0,1,New York,4.0,2011,3,907.0,...,4.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
2,JFK,0.0,"New York, NY",341.0,2,New York,0.0,2011,4,900.0,...,-18.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
3,JFK,0.0,"New York, NY",340.0,3,New York,0.0,2011,5,904.0,...,-21.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
4,JFK,1.0,"New York, NY",336.0,4,New York,76.0,2011,6,1045.0,...,76.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
5,JFK,0.0,"New York, NY",322.0,6,New York,0.0,2011,8,905.0,...,-21.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
6,JFK,0.0,"New York, NY",342.0,2,New York,0.0,2011,11,906.0,...,-15.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1986,BOEING,25
7,JFK,0.0,"New York, NY",315.0,4,New York,0.0,2011,13,907.0,...,-24.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
8,JFK,0.0,"New York, NY",337.0,6,New York,0.0,2011,15,908.0,...,-6.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1986,BOEING,25
9,JFK,0.0,"New York, NY",324.0,7,New York,0.0,2011,16,905.0,...,-40.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24


In [31]:
ret.head()

,level_0,Unnamed: 0,Origin,ArrDel15,OriginCityName,AirTime,DayOfWeek,OriginStateName,ArrDelayMinutes,Year,...,ArrDelay,Month,UniqueCarrier,Diverted,Dest,DestCityName,AirlineID,AIRCRAFT_YEAR,AIRCRAFT_MFR,AIRCRAFT_AGE
0,0,0,JFK,0.0,"New York, NY",362.0,7,New York,1.0,2011,...,1.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1988,BOEING,23
1,1,1,JFK,0.0,"New York, NY",359.0,1,New York,4.0,2011,...,4.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
2,2,2,JFK,0.0,"New York, NY",341.0,2,New York,0.0,2011,...,-18.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
3,3,3,JFK,0.0,"New York, NY",340.0,3,New York,0.0,2011,...,-21.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24
4,4,4,JFK,1.0,"New York, NY",336.0,4,New York,76.0,2011,...,76.0,1,AA,0.0,LAX,"Los Angeles, CA",19805,1987,BOEING,24


In [24]:
ret.to_csv('Big10FlightTable.csv')

In [25]:
ret = []